In [1]:
import os
os.chdir("/tf/src/examples/greetings")

from greetings import Dataset
import tensorflow as tf
import numpy as np
import logging
from greetings import GreetingModel, cfg
from copynet_tf.loss import CopyNetLoss
from copynet_tf.metrics import BLEU
logging.basicConfig(
        level=cfg.LOG_LVL,
        filename=cfg.LOG_FILENAME,
        format='%(message)s')

In [2]:
data = Dataset()

In [3]:
model = GreetingModel()

In [4]:
RNG_SEED = 11
to_gpu = tf.data.experimental.copy_to_device("/gpu:0")
data = data.train.shuffle(
    buffer_size=10000, seed=RNG_SEED, reshuffle_each_iteration=False)
train = data.skip(512).batch(512, drop_remainder=True).apply(to_gpu)
val = data.take(512).batch(512, drop_remainder=True).apply(to_gpu)
with tf.device("/gpu:0"):
    train = train.prefetch(3)
    val = val.prefetch(3)

In [5]:
X, y = next(val.as_numpy_iterator())
print("source idx from source vocab\n", X[0][:3])
print("\nsource idx from target vocab\n", X[1][:3])
print("\ntarget idx from target vocab\n", y[0][:3])
print("\ntarget idx from source vocab\n", y[1][:3])

source idx from source vocab
 [[   2   11   29    4   15    7 2723  773    4    3]
 [   2   22   75    8   14    4    3    0    0    0]
 [   2   18   28    6    5  232 4784    3    0    0]]

source idx from target vocab
 [[ 2  1  1  1  1  1  1  1  1  3]
 [ 2  1 54  8  1  1  3  0  0  0]
 [ 2  1  1  1  1  1  1  3  0  0]]

target idx from target vocab
 [[ 2  4  5  6  7  8  1  1  9  3]
 [ 2  4  5  6  7  8 54  9  3  0]
 [ 2  4  5  6  7  8  1  1  9  3]]

target idx from source vocab
 [[   2    1    1    1    1    8 2723  773    1    3]
 [   2    1    1    1    1    8   75    1    3    0]
 [   2    1    1    1    1    8  232 4784    1    3]]


In [6]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3, clipnorm=cfg.CLIP_NORM),
    loss=CopyNetLoss(),
    metrics=[
        BLEU(ignore_tokens=[0,2,3], ignore_all_tokens_after=3),
        BLEU(ignore_tokens=[0,2,3], ignore_all_tokens_after=3, name='bleu-smooth', smooth=True)])

In [7]:
ckpt = tf.keras.callbacks.ModelCheckpoint(cfg.MODEL_SAVE+"/{epoch:02d}.tf", monitor='val_bleu', save_weights_only=True)
hist = model.fit(train, epochs=5, validation_data=val, shuffle=False, callbacks=[ckpt])

Epoch 1/5
8/8 [==============================] - 5s 684ms/step - loss: 1.2604 - bleu: 0.5563 - bleu-smooth: 0.5563 - val_loss: 0.2107 - val_bleu: 0.8114 - val_bleu-smooth: 0.8115
Epoch 2/5
8/8 [==============================] - 4s 479ms/step - loss: 0.1467 - bleu: 0.8520 - bleu-smooth: 0.8520 - val_loss: 0.0772 - val_bleu: 0.9376 - val_bleu-smooth: 0.9376
Epoch 3/5
8/8 [==============================] - 4s 474ms/step - loss: 0.0471 - bleu: 0.9631 - bleu-smooth: 0.9631 - val_loss: 0.0224 - val_bleu: 0.9855 - val_bleu-smooth: 0.9855
Epoch 4/5
8/8 [==============================] - 4s 474ms/step - loss: 0.0133 - bleu: 0.9934 - bleu-smooth: 0.9934 - val_loss: 0.0101 - val_bleu: 0.9889 - val_bleu-smooth: 0.9889
Epoch 5/5
8/8 [==============================] - 4s 478ms/step - loss: 0.0051 - bleu: 0.9974 - bleu-smooth: 0.9974 - val_loss: 0.0051 - val_bleu: 0.9966 - val_bleu-smooth: 0.9966


In [8]:
def idx2str(pred_y, X):
    ret = []
    vocab_len = model.vocab.get_vocab_size("target")
    for idx in pred_y:
        if idx < vocab_len:
            ret.append(model.vocab.get_token_text(idx, "target"))
        else:
            ret.append(model.vocab.get_token_text(X[idx-vocab_len], "source"))
    return ret

In [9]:
pred = model.predict(train)
pred, pred_proba = pred["predictions"], pred["predicted_probas"]
for i, Xy in enumerate(train.unbatch().take(10)):
    X, y = Xy
    s = model.vocab.inverse_transform(X[0].numpy()[np.newaxis, :], "source")[0]
    t = model.vocab.inverse_transform(y[0].numpy()[np.newaxis, :], "target")[0]
    print(f"Source: {' '.join(s)}\nTarget: {' '.join(t)}\n")
    for j in range(10):
        p = idx2str(pred[i][j], X[0].numpy())
        print(f"Predicted: {' '.join(p)}\tProba: {tf.exp(pred_proba[i][j])}")
    print("")

Source: <S> Call me Alma Poole EOS PAD PAD PAD PAD
Target: <S> Nice to meet you , UNKNOWN UNKNOWN ! EOS

Predicted: Nice to meet you , Alma Poole ! EOS	Proba: 0.9986456036567688
Predicted: Nice to meet you , Alma ! EOS EOS	Proba: 0.00032091891625896096
Predicted: Nice to meet , Alma Poole ! EOS EOS	Proba: 0.00024326627317350358
Predicted: to meet you , Alma Poole ! EOS EOS	Proba: 0.00016099664208013564
Predicted: Nice to meet you Alma Poole ! EOS EOS	Proba: 0.00015887137851677835
Predicted: Nice to you , Alma Poole ! EOS EOS	Proba: 0.00014189528883434832
Predicted: Nice to meet meet you , Alma Poole !	Proba: 6.30374561296776e-05
Predicted: Nice to to meet you , Alma Poole !	Proba: 5.987837357679382e-05
Predicted: Nice to meet you , Alma Poole Nice to	Proba: 5.6703443988226354e-05
Predicted: Nice meet you , Alma Poole ! EOS EOS	Proba: 4.0978316974360496e-05

Source: <S> My friends call me Nathanael EOS PAD PAD PAD
Target: <S> Nice to meet you , UNKNOWN ! EOS PAD

Predicted: Nice to meet

In [10]:
pred = model.predict(val.unbatch().take(50).batch(5, drop_remainder=True))
pred, pred_proba = pred["predictions"], pred["predicted_probas"]
for i, Xy in enumerate(val.unbatch().take(10)):
    X, y = Xy
    s = model.vocab.inverse_transform(X[0].numpy()[np.newaxis, :], "source")[0]
    t = model.vocab.inverse_transform(y[0].numpy()[np.newaxis, :], "target")[0]
    print(f"Source: {' '.join(s)}\nTarget: {' '.join(t)}\n")
    for j in range(10):
        p = idx2str(pred[i][j], X[0].numpy())
        print(f"Predicted: {' '.join(p)}\tProba: {tf.exp(pred_proba[i][j])}")
    print("")

Source: <S> Hi there . It 's Nyla Wais . EOS
Target: <S> Nice to meet you , UNKNOWN UNKNOWN ! EOS

Predicted: Nice to meet you , Nyla Wais ! EOS	Proba: 0.9983201622962952
Predicted: Nice to meet you , Nyla Wais Wais !	Proba: 0.0006818209658376873
Predicted: Nice to meet , Nyla Wais ! EOS EOS	Proba: 0.00019870063988491893
Predicted: Nice to meet you Nyla Wais ! EOS EOS	Proba: 0.00015910154615994543
Predicted: Nice to you , Nyla Wais ! EOS EOS	Proba: 0.00014713183918502182
Predicted: to meet you , Nyla Wais ! EOS EOS	Proba: 0.00012121191684855148
Predicted: Nice to meet you , Nyla Wais . !	Proba: 0.00011916882795048878
Predicted: Nice to to meet you , Nyla Wais !	Proba: 7.628375897184014e-05
Predicted: Nice to meet meet you , Nyla Wais !	Proba: 6.900600419612601e-05
Predicted: Nice meet you , Nyla Wais ! EOS EOS	Proba: 3.1394902180181816e-05

Source: <S> Its Roseann , actually . EOS PAD PAD PAD
Target: <S> Nice to meet you , Roseann ! EOS PAD

Predicted: Nice to meet you , Roseann ! EOS 